In [1]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geomloss import SamplesLoss
import torch
from sklearn import preprocessing

[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


In [2]:
gan_swat_network = pd.read_csv("gan_lstm_swat_network.csv", sep=",").to_numpy()
gan_swat_sensor = pd.read_csv("gan_lstm_swat_sensor.csv", sep=",").to_numpy()
gan_wadi = pd.read_csv("gan_lstm_wadi.csv", sep=",").to_numpy()
gan_wustl = pd.read_csv("gan_lstm_wustl.csv", sep=",").to_numpy()

vae_swat_network = pd.read_csv("vae_swat_network.csv", sep=",").to_numpy()
vae_swat_sensor = pd.read_csv("vae_swat_sensor.csv", sep=",").to_numpy()
vae_wadi = pd.read_csv("vae_wadi_sensor.csv", sep=",").to_numpy()
vae_wustl = pd.read_csv("vae_wustl_network.csv", sep=",").to_numpy()

vae_gan_swat_network = pd.read_csv("vae_gan_swat_network.csv", sep=",").to_numpy()
vae_gan_swat_sensor = pd.read_csv("vae_gan_swat_sensor.csv", sep=",").to_numpy()
vae_gan_wadi = pd.read_csv("vae_gan_wadi_sensor.csv", sep=",").to_numpy()
vae_gan_wustl = pd.read_csv("vae_gan_wustl_network.csv", sep=",").to_numpy()

gan_softmax_swat_network = pd.read_csv("gan_softmax_swat_network_50.csv", sep=",").to_numpy()
gan_softmax_swat_sensor = pd.read_csv("gan_softmax_swat_sensor_50.csv", sep=",").to_numpy()
gan_softmax_wadi = pd.read_csv("gan_softmax_wadi_50.csv", sep=",").to_numpy()
gan_softmax_wustl = pd.read_csv("gan_softmax_wustl_50.csv", sep=",").to_numpy()

In [3]:
swat = preprocessing.normalize(pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/swat_processed.csv", sep=",").to_numpy()[:, 1:])
# swap sport to front
swat[:, [0, 1]] = swat[:, [1, 0]]
# # swap dport to after sport
swat[:, [1, 2]] = swat[:, [2, 1]]
# # swap protocols to after dport
swat[:, [2, 14]] = swat[:, [14, 2]]

wadi = preprocessing.normalize(pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/wadi_attacks.csv", sep=",").to_numpy()[:, 1:])
swat_sensor = preprocessing.normalize(pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/swat_sensor.csv", sep=",").to_numpy()[:, 1:])
wustl = preprocessing.normalize(pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/wustl_attacks.csv", sep=",").to_numpy()[:, 1:])

In [4]:
loss = SamplesLoss("sinkhorn", blur=0.05)

In [5]:
def wasserstein(real, fake):
    loss = SamplesLoss("sinkhorn", blur=0.05)

    num_features = real.shape[1]
    real = torch.from_numpy(real[: fake.shape[0]])
    fake = torch.from_numpy(fake)

    return loss(real, fake)

## GAN

In [6]:
wasserstein(swat, gan_swat_network[:, 1:])

tensor(0.7820, dtype=torch.float64)

In [7]:
wasserstein(swat_sensor, gan_swat_sensor)

tensor(6.7952, dtype=torch.float64)

In [8]:
wasserstein(wadi, gan_wadi)

tensor(13.0911, dtype=torch.float64)

In [9]:
wasserstein(wustl, gan_wustl)

tensor(1.9241, dtype=torch.float64)

## VAE

In [10]:
wasserstein(swat, vae_swat_network[:5000, 1:])

tensor(0.6788, dtype=torch.float64)

In [11]:
wasserstein(swat_sensor, vae_swat_sensor)

tensor(7.8900, dtype=torch.float64)

In [12]:
wasserstein(wadi, vae_wadi)

tensor(11.7697, dtype=torch.float64)

In [13]:
wasserstein(wustl, vae_wustl[:5000])

tensor(1.5986, dtype=torch.float64)

## VAE+GAN

In [14]:
wasserstein(swat, vae_gan_swat_network[:, 1:])

tensor(0.7038, dtype=torch.float64)

In [15]:
wasserstein(swat_sensor, vae_gan_swat_sensor.reshape(2640, 28))

tensor(4.0623, dtype=torch.float64)

In [16]:
wasserstein(wadi, vae_gan_wadi.reshape(1996, 69))

tensor(8.4115, dtype=torch.float64)

In [17]:
wasserstein(wustl, vae_gan_wustl.reshape(17404, 27)[:5000])

tensor(1.9475, dtype=torch.float64)

## GAN Softmax

In [18]:
wasserstein(swat, gan_softmax_swat_network)

tensor(2.5661e+08, dtype=torch.float64)

In [19]:
wasserstein(swat_sensor, gan_softmax_swat_sensor)

tensor(6.1560, dtype=torch.float64)

In [20]:
wasserstein(wadi, gan_softmax_wadi)

tensor(9.6024, dtype=torch.float64)

In [21]:
wasserstein(wustl, gan_softmax_wustl)

tensor(1.4338e+08, dtype=torch.float64)